In [1]:
import pandas as pd

import tensorflow as tf
import json
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

from cell_division.nets.transfer_learning import CNN
from auxiliary.data.dataset_cell import CellDataset
from auxiliary.data.dataset_unlabeled import UnlabeledDataset
from auxiliary import values as v
from auxiliary.utils.colors import bcolors as c
from auxiliary.utils import visualizer as vis

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D
from cell_division.nets.custom_layers import (
    w_cel_loss, 
    focal_loss,
    ExtendedLSEPooling,
    extended_w_cel_loss,
    LSEPooling
)

from cell_division.nets.cam import GradCAM, overlay_heatmap, CAM, GradCAMpp
from cell_division.semi_supervised import semi_supervised_learning as SSL

# GPU config
from auxiliary.utils.timer import LoadingBar
from auxiliary.gpu.gpu_tf import (
    increase_gpu_memory, 
    set_gpu_allocator, 
    clear_session
)

increase_gpu_memory()
# set_gpu_allocator()

ValueError: Cannot set memory growth on device when virtual devices configured

In [2]:
img_dir = v.data_path + 'CellDivision/images_nuclei/'
img_dir_unlabeled = v.data_path + 'CellDivision/images_unlabeled/'

label_train_dir = v.data_path + 'CellDivision/undersampled/train.csv'
label_test_dir = v.data_path + 'CellDivision/undersampled/test.csv'
label_val_dir = v.data_path + 'CellDivision/undersampled/val.csv'

INPUT_SHAPE = (100, 100, 3)
BATCH_SIZE = 128

In [3]:
train_generator = CellDataset(
    img_dir, 
    label_train_dir,
    batch_size=BATCH_SIZE,
    resize=INPUT_SHAPE[:2]
)

val_generator = CellDataset(
    img_dir, 
    label_val_dir,
    batch_size=BATCH_SIZE,
    resize=INPUT_SHAPE[:2]
)

test_generator = CellDataset(
    img_dir, 
    label_test_dir,
    batch_size=BATCH_SIZE,
    resize=INPUT_SHAPE[:2]
)

unlabeled_generator = UnlabeledDataset(
    img_dir_unlabeled,
    batch_size=1,
    resize=INPUT_SHAPE[:2]
)

In [4]:
model = CNN(
    base=tf.keras.applications.VGG16,
    input_shape=INPUT_SHAPE,
    n_classes=3
)
model.build_top(activation='softmax', b_type='CAM', pooling=ExtendedLSEPooling)
model.compile(
    lr=.001,
    loss=extended_w_cel_loss()
)

In [10]:
model = SSL(
    model, 
    train_generator, val_generator, test_generator,
    unlabeled_generator,
    max_iter=10,
    verbose=1
)
model.model.save('../models/cellular_division_models/vgg16_semi.h5')

Iteration: 1
	Pre-training...
Epoch 1/100
7/7 [==============================] - 13s 2s/step - loss: 0.5812 - auc: 0.9683 - val_loss: 1.0401 - val_auc: 0.8645 - lr: 1.0000e-06
Epoch 2/100
7/7 [==============================] - 12s 2s/step - loss: 0.8229 - auc: 0.9451 - val_loss: 1.0110 - val_auc: 0.8792 - lr: 1.0000e-06
Epoch 3/100
7/7 [==============================] - 13s 2s/step - loss: 0.6857 - auc: 0.9632 - val_loss: 0.9820 - val_auc: 0.8948 - lr: 1.0000e-06
Epoch 4/100
7/7 [==============================] - ETA: 0s - loss: 0.7221 - auc: 0.9533
Epoch 4: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
7/7 [==============================] - 16s 2s/step - loss: 0.7221 - auc: 0.9533 - val_loss: 0.9537 - val_auc: 0.9066 - lr: 1.0000e-06
Epoch 5/100
7/7 [==============================] - 14s 2s/step - loss: 0.7073 - auc: 0.9581 - val_loss: 0.9256 - val_auc: 0.9173 - lr: 1.0000e-07
Epoch 6/100
7/7 [==============================] - 13s 2s/step - loss: 0.5832 - auc: 0.

2024-08-14 13:26:08.803886: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/CellDivision/images_unlabeled/0402_E2_45799_1.tif'
Traceback (most recent call last):

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 830, in wrapped_generator
    for 

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: '/run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/CellDivision/images_unlabeled/0402_E2_45799_1.tif'
Traceback (most recent call last):

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/imarcoss/mambaforge/envs/py310ml/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 830, in wrapped_generator
    for data in

In [ ]:
model.model.evaluate(test_generator)

In [ ]:
pred = model.model.predict(test_generator).round().astype(int)

print(
    classification_report(
        test_generator.img_labels, [test_generator.oh2class(p) for p in pred], 
        target_names=test_generator.CLASS_NAMES,
        zero_division=0
    )
)

In [ ]:
cf_matrix = confusion_matrix(
   test_generator.img_labels, [test_generator.oh2class(p) for p in pred]
)

plt.figure(figsize=(12, 12))
vis.plot_confusion_matrix(cf_matrix)